# Lecture 9: Matrix decompositions and how we compute them

## Syllabus
**Week 1:** Matrices, vectors, matrix/vector norms, scalar products & unitary matrices  
**Week 2:** TAs-week (Strassen, FFT, a bit of SVD)  
**Week 3:** Matrix ranks, singular value decomposition, linear systems, eigenvalues  
**Week 4:** Matrix decompositions: QR, LU, SVD + test + structured matrices start

## Recap of the previous lecture
- Eigenvectors and eigenvalues
- Characteristic polynomial and why it is a bad idea
- Power method: $x := Ax$
- Gershgorin theorem
- Schur theorem: $A = U T U^*$ 
- Normal matrices: $A^* A = A A^*$
- A brief whiteboard illustration of how to compute the Schur form: QR-algorithm

## Today lecture
Today we will talk about **matrix factorizations**

- LU decomposition and Gaussian elimination
- QR decomposition and Gram-Schmidt algorithm
- Schur decomposition and QR-algorithm

We already introduced it some time ago, but now we are going to do it in more details.  

These are the basic **matrix factorizations** in numerical linear algebra.

## General concept of matrix factorization

In numerical linear algebra we need to solve different tasks, for example:

- Solve linear systems $Ax = f$
- Compute eigenvalues / eigenvectors
- Compute singular values / singular vectors
- Compute inverses, even sometimes determinants 
- Compute **matrix functions** like $\exp(A), \cos(A)$.

In order to do this, we replace the matrix as a sum and/or product of matrices with **simpler structure**,  
such that we can solve abovementioned tasks faster / more stable form.

What is a **simpler structure**?


## What is a simpler structure
We already encountered several classes of matrices **with structure**. 

In dense matrix business, the most important classes are

**unitary matrices** ($U^* U = I$, why?), **upper/lower triangular matrices **, **diagonal matrices**

## Other classes of matrices

For **sparse matrices** the sparse constraints are often included in the factorizations.  

For **Toeplitz matrices** an important class of matrices is the matrices with **low displacement rank**, 

which is based on the low-rank matrices. 

The class of **low-rank matrices** and **block low-rank matrices** is everywhere.

## Plan
The plan for todays lecture is to discuss the decompositions one-by-one and point out:
- How to compute a particular decomposition
- When the decomposition exists 
- What is done in "real life"

## Decompositions we want to discuss today
- LU factorization
- Positive definite matrices and Cholesky factorization
- QR-decomposition and Gram-Schmidt algorithm
- 1 slide about the SVD (more tomorrow)

## What is the LU factorization

LU factorization is the representation of a given matrix $A$ as a product  
of a **lower triangular matrix** $L$ and an **upper triangular** matrix $U$:

$$A = LU$$

This factorization is **non-unique**, so it is typical to require that the matrix $L$ has ones on the diagonal.

Does it always exist?

**Main goal** of the LU decomposition is to solve linear systems, because

$$
    A^{-1} f = (L U)^{-1} f = U^{-1} L^{-1} f, 
$$

and this reduces to the solution of two linear systems 
$$
     L y = f, 
$$
and 
$$
   U x = y.
$$

Solving linear systems with **triangular matrices** is **easy**: you just solve the equation with one unknown, put into the second equation, and so on.  

The computational cost of such algorithm is $\mathcal{O}(n^2)$ arithmetic operations, once $LU$ factorization is known. 

Note that the **inverse of triangular matrix** is a triangular matrix.

In [2]:
%matplotlib inline
import numpy as np
import sympy
from sympy.matrices import Matrix
import IPython
from sympy.interactive.printing import init_printing
init_printing(use_latex=True)
n = 5
w = Matrix(n, n, lambda i, j: 1/(i + j +  sympy.Integer(1)/2))
L, U, tmp = w.LUdecomposition()
#Generate the final expression
#fn = u'%s \\times %s = %s' % (sympy.latex(L), sympy.latex(U), sympy.latex(w))
#L * U - w
L

⎡ 1    0    0   0   0⎤
⎢                    ⎥
⎢1/3   1    0   0   0⎥
⎢                    ⎥
⎢1/5  6/7   1   0   0⎥
⎢                    ⎥
⎢          15        ⎥
⎢1/7  5/7  ──   1   0⎥
⎢          11        ⎥
⎢                    ⎥
⎢     20   210  28   ⎥
⎢1/9  ──   ───  ──  1⎥
⎣     33   143  15   ⎦


## The classical LU-decomposition algorithm
The simplest way to implement LU-decomposition is the following 3-cycle

In [3]:
from sympy.interactive.printing import init_printing
init_printing(use_latex=True)
L = Matrix(n, n, lambda i, j: 0)
U = Matrix(n, n, lambda i, j: 0)
a = w.copy() #Our matrix
for k in xrange(n): #Eliminate one row
    L[k, k] = 1
    for i in xrange(k+1, n):
        L[i, k] = a[i, k] / a[k, k]
        for j in xrange(k+1, n):
            a[i, j] = a[i, j] - L[i, k] * a[k, j]
    for j in xrange(k, n):
        U[k, j] = a[k, j]

## Existence of the LU-decomposition
The LU-decomposition algorithm does not fail  

if **we do not divide by zero** at every step.

When it is so, for which class of matrices?

It is true for **strictly regular matrices**.

## Strictly regular matrices and LU decomposition 
A matrix $A$ is called strictly regular, if all of its **principal minors**  
(i.e, submatrices in the first $k$ rows and $k$ columns) are non-singular. 

In this case, there always exists a LU-decomposition.  The reverse is also true (check!).

**Proof**. If there is a LU-decomposition, then the matrix is strictly regular 

This follows from the fact that to get a minor you multiply a corresponding submatrix of $L$ by a corresponding submatrix of $U$, and they are non-singular (since non-singularity of triangular matrices is equivalent to the fact that their diagonal elements are not equal to zero).

The other way can be proven by **induction**. Suppose that we know that for all $(n-1) \times (n-1)$ matrices will non-singular minors LU-decomposition exists.

Then, consider the block form
$$
    A = \begin{bmatrix}
          a & c^{\top} \\
          b & D
    \end{bmatrix},
$$
where $D$ is $(n-1) \times (n-1)$.

Then we do "block elimination": 

$$
     \begin{bmatrix}
     1 & 0 \\
     -z & I
     \end{bmatrix}
     \begin{bmatrix}
     a & c^{\top} \\
     b & D 
     \end{bmatrix}=
     \begin{bmatrix}
     a & c^{\top} \\
     0 & A_1
     \end{bmatrix},
$$

where $z = \frac{b}{a}, \quad A_1 = D - \frac{1}{a} b c^{\top}$.  
We can show that $A_1$ is also strictly regular, thus it has (by induction) the LU-decomposition:  
$$
   A_1 = L_1 U_1, 
$$
And that gives the $LU$ decomposition of the original matrix.


## When LU fails
What happens, if the matrix is not strictly regular (or the **pivots** in the Gaussian elimination are really small?). 

There is classical $2 \times 2$ example of a matrix with a bad LU decomposition.  

The matrix we look at is  
$$
    A = \begin{pmatrix}
    \varepsilon & 1 \\
    1 & 1 
    \end{pmatrix}
$$

If $\varepsilon$ is sufficiently small, we **might** fail.

Let us do some demo here.

In [45]:
import numpy as np
eps = 1.12e-16
a = [[eps, 1],[1.0,  1]]
a = np.array(a)
a0 = a.copy()
n = a.shape[0]
L = np.zeros((n, n))
U = np.zeros((n, n))
for k in xrange(n): #Eliminate one row   
    L[k, k] = 1
    for i in xrange(k+1, n):
        L[i, k] = a[i, k] / a[k, k]
        for j in xrange(k+1, n):
            a[i, j] = a[i, j] - L[i, k] * a[k, j]
    for j in xrange(k, n):
        U[k, j] = a[k, j]

print 'L * U - A:', np.dot(L, U) - a0
L

L * U - A: [[ 0.  0.]
 [ 0.  0.]]


array([[  1.00000000e+00,   0.00000000e+00],
       [  8.92857143e+15,   1.00000000e+00]])

## The concept of pivoting
We can do pivoting, i.e. permute rows and columns to maximize $A_{kk}$ that we divide over.  
The simplest but effective strategy is the **row pivoting**: at each step, select the index that is maximal in modulus, and put it onto the diagonal. 

It gives us the decomposition 

$$A = P L U,$$
where $P$ is a **permutation matrix**.


What makes **row pivoting** good?  

It is made good by the fact that  

$$
   | L_{ij}|<1,
$$
but the elements of $U$ can grow, up to $2^n$! (in practice, this is very rarely encountered).

Can you come up with a matrix where the elements of $U$ grow as much as possible.

## Positive definite matrices

Strictly regular matrices have LU-decomposition. 

An important **subclass** of strictly regular matrices is the class of **Hermitian positive definite matrices**

## Positive definite matrices(2)
**Definition:** A matrix is called <font color='red'> positive definite </font> if for any $x: \Vert x \Vert \ne 0$ we have

$$(Ax, x) > 0.$$

## Symmetric positive definite matrices

**Statement:** A Hermitian positive definite matrix $A$ is strictly regular and has **Cholesky factorization** of the form 

$$A = RR^*,$$

where $R$ is upper triangular.

Let us try to prove this fact (on the whiteboard).

The Cholesky factorization is **always stable**.

It is sometimes referred to as "square root" of the matrix.

## Summary on the LU for dense matrices
- Principal submatrices non-singular -- there exists LU
- Pivoting is needed to avoid error growth
- Complexity is $\mathcal{O}(n^3)$ for the factorization step (can be speeded by using Strassen ideas) and $\mathcal{O}(n^2)$ for a solve
- For SPD matrices Cholesky factorization is the method of choice (stability, twice less memory to store)

## QR-decomposition
The next decomposition: **QR** decomposition. Again from the name it is clear that a matrix is represented as a product  
$$
    A = Q R, 
$$
where $Q$ is an **orthogonal (unitary)** matrix and $R$ is **upper triangular**.  

The matrix sizes: $Q$ is $n \times m$, $R$ is $m \times m$.

QR-factorization is defined for **any rectangular matrix**.

## QR-decomposition: applications

This algorithm plays a crucial role in many problems:
- Computing orthogonal bases in a linear space
- Used in the preprocessing step for the SVD
- QR-algorithm for the computation of eigenvectors and eigenvalues (one of the 10 most important algorithms of the 20th century) is    based on the QR-decomposition

## Theorem 
Every rectangular $n \times m$ matrix, $n \geq m$ matrix has a QR-decomposition. 
There are several ways to prove it and compute it.

- (theoretical) Using the Gram matrices and Cholesky factorization
- (geometrical) Using the Gram-Schmidt orthogonalization
- (practical) Using Householder/Givens transformations 



## Mathematical way
If we have the representation of the form
$$A = QR,$$
then $A^* A = R^* R$, the matrix $A^* A$ is called **Gram matrix**, and its elements are scalar products of the columns of $A$.  

## Math way: full column rank

Assume that $A$ has **full column rank**. Then, 

It is simple to show that $A^* A$ is positive definite:

$$
   (A^* A y, y) = (Ay, Ay)^2 = \Vert Ay \Vert  \geq 0.
$$

Therefore, $A^* A = R^* R$ always exists.

Then the matrix $A R^{-1}$ is unitary:  

$$
   (A R^{-1})^* (AR^{-1})= R^{-*} A^* A R^{-1} = R^{-*} R^* R R^{-1} = I.
$$


## Rank-deficient case
When an $n \times m$ matrix does not have full column rank, it is said to be **rank-deficient**.  

The QR-decomposition, however, also exists.

For any **rank-deficient matrix** there is a sequence of full-column rank matrices $A_k$ such that $A_k \rightarrow A$.

Each $A_k$ can be decomposed as $A_k = Q_k R_k$. 

The set of all unitary matrices is compact, thus there exists a converging subsequence $Q_{n_k} \rightarrow Q$,

and $Q^* A_k \rightarrow Q^* A = R$, which is triangular.

## QR-decomposition and Gram matrices
So, the simplest way to compute QR-decomposition is then

$$A^* A = R^* R,$$

(Cholesky factorization)

$$Q = A R^{-1}.$$

It is a **bad idea** for numerical stability. Let us do some demo (for a submatrix of the Hilbert matrix).

In [47]:
import numpy as np
n = 20
r = 8
#a = np.random.randn(n, r)
a = [[1.0/(i+j+0.5) for i in xrange(r)] for j in xrange(n)]
a = np.array(a)
q, Rmat = np.linalg.qr(a)
e = np.eye(r)
print 'Built-in QR orth', np.linalg.norm(np.dot(q.T, q) - e)
gram_matrix = a.T.dot(a)
Rmat1 = np.linalg.cholesky(gram_matrix)
q1 = np.dot(a, np.linalg.inv(Rmat1.T))
print 'Via Gram matrix:', np.linalg.norm(np.dot(q1.T, q1) - e)

Built-in QR orth 8.77419464402e-16
Via Gram matrix: 0.265317882364


## Second way: Gram-Schmidt orthogonalization
QR-decomposition is a mathematical way of writing down the Gram-Schmidt orthogonalization process.  
Given a sequence of vectors $a_1, \ldots, a_m$ we want to find orthogonal basis $q_1, \ldots, q_m$ such that every $a_i$ is a linear combination of such vectors.  

**Gram-Schmidt:**
1. $q_1 := a_1/\Vert a_1 \Vert$
2. $q_2 := a_2 - (a_2, q_1) q_1, \quad q_2 := q_2/\Vert q_2 \Vert$
3. $q_3 := a_3 - (a_3, q_1) q_1 - (a_3, q_2) q_2, \quad q_2 := q_3/\Vert q_3 \Vert$
4. And go on  

Note that the transformation from $Q$ to $A$ has triangular structure, since from the $k$-th vector we subtract only the previous ones.

## Gram-Schmidt is unstable
Gram-Schmidt can be very unstable (i.e., the produced vectors will be not orthogonal, especially if $q_k$ has small norm).  
This is called **loss of orthogonality**.  

There is a remedy, called **modified Gram-Schmidt** method. 
Instead of doing 

$$q_k := a_k - (a_k, q_1) q_1 - \ldots - (a_k, q_{k-1}) q_{k-1}$$

we do it step-by-step. First we set $q_k := a_k$ and orthogonalize sequentially:

$$
   q_k := q_k - (q_k, q_1)q_1, \quad q_k := q_{k} - (q_k,q_2)q_2, \ldots
$$

In exact arithmetic, it is the same. In floating point it is absolutely different!

Note that the complexity is $\mathcal{O}(nm^2)$ operations

## QR-decomposition: the (almost) practical way

If $A = QR$, then  
$$
R = Q^* A,
$$
and we need to find a certain orthogonal matrix $Q$ that brings a matrix into orthogonal form.  
For simplicity, we will look for an $n \times n$ matrix such that
$$
   Q^* A = \begin{bmatrix}
   * & * & *  \\
   0 & * & * \\
   0 & 0 & * \\
   &0_{(n-m) \times m}
   \end{bmatrix}
$$



We will do it column-by-column.  
First, we find a Householder matrix $H_1 = (I - 2 uu^{\top})$ such that (we illustrate on a $4 \times 3$ matrix)

$$
   H_1 A = \begin{bmatrix}
    * & * & * \\
    0 & * & * \\
    0 & * & * \\
    0 & * & *
   \end{bmatrix}
$$

Then, 
$$
   H_2 H_1 A = \begin{bmatrix}
    * & * & * \\
    0 & * & * \\
    0 & 0 & * \\
    0 & 0 & *
   \end{bmatrix},
$$
where
$$
  H_2 = \begin{bmatrix}
  1 & 0 \\
  0 & H'_2, 
  \end{bmatrix}
$$
and $H'_2$ is a $3 \times 3$ Householder matrix.

Finally, 
$$
   H_3 H_2 H_1 A = \begin{bmatrix}
    * & * & * \\
    0 & * & * \\
    0 & 0 & * \\
    0 & 0 & 0
   \end{bmatrix},
$$

You can try to implement by yourself, it is simple.

## QR-decomposition: real life

In reality, since this is a dense matrix factorization, you should implement the algorithm in terms of blocks.  

Instead of using Householder transformation, we use **block Householder** transformation of the form

$$H = (I - 2UU^*), $$

where $U^* U = I$.

This allows us to use BLAS-3 operations. 

## Rank-revealing QR-decomposition

The QR-decomposition can be also used to compute the (numerical) rank of the matrix.  

It is done via so-called **rank-revealing** factorization.  

It is based on the representation

$$P A = Q R, $$

where $P$ is the permutation matrix (it permutes columns), and  $R$ has the block form


$$R = \begin{bmatrix} R_{11} & R_{12} \\ 0 & R_{22}\end{bmatrix},$$

and the norm of $R_{22}$ is **small**, so you can find the numerical rank by looking at it.

## Summary

LU and QR decompositions can be computed using **direct methods** in finite amount of operations.

What about Schur form and SVD?  

They can not be computed by **direct methods** they can only be computed by **iterative methods**

## Schur form

Every matrix can be written as a product

$$A = U T U^*,$$

with triangular $T$ and unitary $U$

and this decomposition gives eigenvalues of the matrix.


## QR-algorithm
The QR algorithm (Kublanovskaya and Francis independently proposed it in 1961).   



<font color='red'> Do not **mix** QR algorithm and QR decomposition. </font>

QR-decomposition is the representation of a matrix, whereas QR algorithm uses QR decomposition computes the eigenvalues!


## Way to QR-algorithm

Consider the equation

$$A = Q T Q^*,$$

and rewrite it in the form

$$
   Q T = A Q.
$$

## Derivation of the QR-algorithm as fixed-point
Then we can write down the iterative process 

$$
    Q_{k+1} R_{k+1} = A Q_k, \quad Q_{k+1}^* A = R_{k+1} Q^*_k
$$

Introduce

$$A_k = Q^* _k A Q_k = Q^*_k Q_{k+1} R_{k+1} = \widehat{Q}_k R_{k+1}$$.

and the new approximation reads

$$A_{k+1} = Q^*_{k+1} A Q_{k+1} = \widehat{Q}^*_k A_k \widehat{Q}_k = R_{k+1} \widehat{Q}_k.$$


## Final formulas
The final formulas are then written in the form
$$
   A_0 = A, \quad A_k = Q_k R_k, \quad A_{k+1} = R_k Q_k.
$$

The matrices $A_{k}$ are unitary similar, and $A_k \rightarrow T$.  

The complexity of each step is $\mathcal{O}(n^3)$.

In [49]:
import numpy as np
n = 4
a = [[1.0/(i + j + 0.5) for i in xrange(n)] for j in xrange(n)]
niters = 100
for k in xrange(niters):
    q, rmat = np.linalg.qr(a)
    a = rmat.dot(q)
    print 'current a:'
    print a


current a:
[[  2.40047183e+00   1.43485636e-01   4.99605047e-03  -5.56291523e-05]
 [  1.43485636e-01   3.59286592e-01   1.60534687e-02  -1.94225770e-04]
 [  4.99605047e-03   1.60534687e-02   1.60692682e-02  -2.80885670e-04]
 [ -5.56291523e-05  -1.94225770e-04  -2.80885670e-04   2.40684296e-04]]
current a:
[[  2.41031150e+00   2.09437993e-02   3.18722176e-05   5.45279213e-09]
 [  2.09437993e-02   3.50196113e-01   6.87806955e-04   1.28079871e-07]
 [  3.18722176e-05   6.87806955e-04   1.53250849e-02   4.17732654e-06]
 [  5.45279205e-09   1.28079871e-07   4.17732654e-06   2.35678648e-04]]
current a:
[[  2.41051991e+00   3.04114601e-03   2.02621961e-07  -5.33227597e-13]
 [  3.04114601e-03   3.49989111e-01   3.00995528e-05  -8.62074652e-11]
 [  2.02621961e-07   3.00995528e-05   1.53236760e-02  -6.42429491e-08]
 [ -5.33147641e-13  -8.62074827e-11  -6.42429491e-08   2.35677492e-04]]
current a:
[[  2.41052431e+00   4.41545673e-04   1.28806658e-09   1.32059800e-16]
 [  4.41545673e-04   3.4998472

## Convergence and complexity of the QR-algorithm
The convergence of the QR-algorithm is from **largest eigenvalues** to the smallest.

At least 2-3 iterations is needed for an eigenvalue. 

Each step is one QR-factorization and one matrix-by-matrix product. $\mathcal{O}(n^3)$ complexity.

It means, $\mathcal{O}(n^4)$ complexity? 

Fortunately, not. 

We can also speedup the QR-algorithm by using **shifts**.

## A few words about the SVD

And the last, but not the least, is the **singular value decomposition** of matrices.

$$A = U \Sigma V^*.$$

We can compute via eigendecomposition of 

$$A^* A = U^* \Sigma^2 U,$$

but it is a **bad idea** (c.f. Gram matrices)

We will discuss methods for computing SVD later (there will a special SVD lecture).


## Summary of todays lecture

- LU decomposition and Gaussian elimination
- QR decomposition and Gram-Schmidt algorithm, reduction to a simpler form by Householder transformations
- Schur decomposition and QR-algorithm



## Next lecture
- No Test
- SVD: how to compute it, what are the applications, computing best rank-$r$ approximation

##### Questions?

In [42]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()